In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime as dt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.preprocessing import LabelBinarizer

In [2]:

MODEL_PATH = "models/convlstm_large_model_Datetime_2022_10_20__22_26_06__loss_0.533180296421051_acc_0.9062302112579346.h5"
SCALER_MAX = 1294.0

FEATURES_PATH = "data/my_20_sec_features-AWS_trial_NO_GUI-2023_03_13__20_29_32.npy"
LABELS_PATH = "data/my_20_sec_labels-AWS_trial_NO_GUI-2023_03_13__20_29_32.npy"

SITES_PATH = "VPN_client_scripts/urls/top_100.csv"

SEED = 42

In [3]:
model = tf.keras.models.load_model(MODEL_PATH)

2023-03-13 20:58:30.061247: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
sites_df = pd.read_csv(SITES_PATH, names=["ranking", "site"])
sites = list(sites_df.site)
len(sites)

100

In [5]:
with open(FEATURES_PATH, 'rb') as f:
    features = np.load(f)

with open(LABELS_PATH, 'rb') as f:
    labels = np.load(f)

In [6]:
for i, label in enumerate(labels):
  if label not in sites:
    labels[i] = "Other"

In [7]:
len(np.unique(labels))

88

In [8]:
label_binariser = LabelBinarizer()
label_binariser.fit(sites)
labels_encoded = label_binariser.transform(labels)
# labels_encoded = label_binariser.fit_transform(labels)
labels_encoded.shape

(2135, 99)

In [13]:
count = 0
for label in np.unique(labels, axis=0):
  print(label)
  count += 1

print(f"Num nuique = {count}")

3dmgame.com
Other
adobe.com
aliexpress.com
alipay.com
amazon.com
amazonaws.com
apple.com
bilibili.com
bing.com
booking.com
canva.com
chaturbate.com
chsi.com.cn
cnblogs.com
deepl.com
discord.com
disneyplus.com
douyu.com
duckduckgo.com
dzen.ru
ebay.com
etsy.com
facebook.com
fandom.com
flipkart.com
freepik.com
github.com
google.com
hao123.com
hbomax.com
huya.com
ilovepdf.com
imdb.com
imgur.com
indeed.com
instagram.com
instructure.com
jd.com
linkedin.com
live.com
mail.ru
mediafire.com
microsoft.com
msn.com
myshopify.com
naver.com
netflix.com
office.com
ok.ru
onlyfans.com
paypal.com
pinterest.com
pornhub.com
primevideo.com
qq.com
quora.com
rakuten.co.jp
reddit.com
roblox.com
sina.com.cn
so.com
sogou.com
sohu.com
soso.com
spankbang.com
spotify.com
stackoverflow.com
stripchat.com
t.me
telegram.org
tiktok.com
tradingview.com
twitch.tv
twitter.com
udemy.com
vk.com
weibo.com
whatsapp.com
wikipedia.org
wordpress.com
xhamster.com
xnxx.com
xvideos.com
yahoo.com
yandex.ru
youtube.com
zoom.us
Num nui

In [9]:
features.shape

(2135, 200, 150, 2)

In [10]:
features_scaled = features/SCALER_MAX

In [11]:
features_scaled.max()

2.1993817619783615

In [12]:
model_evaluation_history = model.evaluate(features_scaled, labels_encoded)

ValueError: in user code:

    File "/Users/michaelkearney/.local/share/virtualenvs/UNB_Datasets-qeLdalLc/lib/python3.9/site-packages/keras/engine/training.py", line 1525, in test_function  *
        return step_function(self, iterator)
    File "/Users/michaelkearney/.local/share/virtualenvs/UNB_Datasets-qeLdalLc/lib/python3.9/site-packages/keras/engine/training.py", line 1514, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/michaelkearney/.local/share/virtualenvs/UNB_Datasets-qeLdalLc/lib/python3.9/site-packages/keras/engine/training.py", line 1507, in run_step  **
        outputs = model.test_step(data)
    File "/Users/michaelkearney/.local/share/virtualenvs/UNB_Datasets-qeLdalLc/lib/python3.9/site-packages/keras/engine/training.py", line 1473, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/michaelkearney/.local/share/virtualenvs/UNB_Datasets-qeLdalLc/lib/python3.9/site-packages/kera

In [ ]:
def plot_results(model):
    fig, ax = plt.subplots(figsize=(30, 30))
    y_true = np.argmax(labels_test, axis=1)
    predictions =  np.argmax(model.predict(features_test), axis=1)
    cm = confusion_matrix(y_true, predictions, normalize='true')
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm,
        display_labels=[label_binariser.classes_[x] for x in range(unique_y.min(), len(unique_y))]
        )
    disp.plot(xticks_rotation=270, values_format=".1f", ax=ax)
    plt.show()
    print(classification_report(y_true, predictions))

In [ ]:
plot_results(model)